In [29]:
import yfinance as yf
import numpy as np
import pandas as pd
import os
from datetime import datetime
from concurrent import futures
from pandas import DataFrame
import pandas_datareader.data as web
import pymongo

In [78]:
import requests_cache
import time

In [44]:
#bitcoin?
bitdata = pd.read_csv('bitindex.csv')
#bitdata.head(5)

In [30]:
# Connect to MongoDB (change connection string if needed)
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["stock_data"]
collection = db["price_data"]

In [131]:
#DANGER THIS DELETES THE WHOLE DATABASE
#collection.delete_many({})

In [ ]:
#rate limit is 2000 requests per hour

In [79]:
# Set up caching
requests_cache.install_cache('yfinance.cache', expire_after=3600)  # expire after 1 hour

In [81]:
# Function to download and store stock data
def download_and_store_stock_data(ticker, start_date, end_date):
    print(f"Downloading data for {ticker}...")
    try:
        # Fetching data using yfinance with caching
        data = yf.download(ticker, start=start_date, end=end_date, interval='1m')
        
        if data.empty:
            print(f"No data found for {ticker}")
            return
        
        # Convert DataFrame to dictionary and add ticker information
        data_dict = data.reset_index().to_dict(orient="records")
        for record in data_dict:
            record['ticker'] = ticker
        
        # Check if data already exists in the database
        existing_data = list(collection.find({
            'ticker': ticker,
            'Datetime': {
                '$gte': start_date,
                '$lt': end_date
            }
        }))

        if existing_data:
            print(f"Data for {ticker} from {start_date} to {end_date} already exists in MongoDB.")
        else:
            collection.insert_many(data_dict)
            print(f"Data for {ticker} from {start_date} to {end_date} stored in MongoDB.")
    
    except Exception as e:
        print(f"Failed to download data for {ticker}. Error: {str(e)}")
    finally:
        # Simple rate limiting: sleep for 2.5 seconds
        time.sleep(2.5)

In [ ]:
#yahoo finance caps me at 1 minute interval. gotta pay for tick by tick numbers

In [83]:
# Download data for a specific stock as an example
# download_and_store_stock_data('AAPL', '2023-10-23', '2023-10-24')

In [85]:
query = {"ticker": "AAPL"}
data = list(collection.find(query))
df = pd.DataFrame(data)
#df.head(5)

In [39]:
	""" list of s_anp_p companies """
	s_and_p = ['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 
		   'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT',
		   'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM',
		   'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BRK.B',
		   'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW',
		   'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY',
		   'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX',
		   'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA',
		   'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA',
		   'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL',
		   'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EVRG', 'ES', 'RE', 'EXC', 'EXPE',
		   'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FMC', 'F', 'FTNT', 'FTV',
		   'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GL', 'GPN', 'GS',
		   'GWW', 'HAL', 'HBI', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM',
		   'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'INTC', 'ICE', 'IBM', 'IP', 'IPG', 'IFF', 'INTU',
		   'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB',
		   'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 
		   'LMT', 'L', 'LOW', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT',
		   'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI',
		   'NDAQ', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE',
		   'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'OKE', 'ORCL', 'OGN', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 
		   'PENN', 'PNR', 'PBCT', 'PEP', 'PKI', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 
		   'PRU', 'PTC', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 
		   'RMD', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS',
		   'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL',
		   'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TWTR', 'TYL', 'TSN', 'UDR',
		   'ULTA', 'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UHS', 'VLO', 'VTR', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VIAC', 
		   'VTRS', 'V', 'VNO', 'VMC', 'WRB', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WU', 'WRK', 'WY', 
		   'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']

In [45]:
SP500_symbols = [] #valid only
# IF I HARD CODE IT THIS WAY THIS CODE WILL NOT BE FLEXIBLE IF THEY MODIFY THE STOCKS IN THE INDEX

def validate_and_add_ticker(ticker):
    try:
        # Attempt to fetch a small amount of data to validate the ticker
        data = yf.download(ticker, period='1d', interval='1m')
        
        if not data.empty:
            #print(f"{ticker} is a valid ticker symbol.")
            SP500_symbols.append(ticker)
        else:
            print(f"{ticker} is not a valid ticker symbol or there is no available data.")
            
    except Exception as e:
        print(f"Failed to fetch data for {ticker}. Error: {str(e)}")

In [47]:
# DO NOT RUN THIS AGAIN UNLESS YOU NEED TO AND DO NOT FUCK WITH THE SP500_SYMBOLS VARIABLE // THIS COST A LOT
# OF MY API RATE LIMIT!

#for ticker in s_and_p:
#    validate_and_add_ticker(ticker)

In [89]:
print(SP500_symbols)

['MMM', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BAC', 'BBWI', 'BAX', 'BDX', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 

In [43]:
'''
- i think i should do a 30 second data step for one day? or a few days? 
- and then i need to 'extract features' to figure out what actualy matters here
- volume? price jumps? a mix of both? 
- relation to other stocks?
- and like, does it have to steadily increase for 2 minutes before we buy? make time a variable i can choose
- also rate of jump in volume and rate of jump in price
- when to sell? when the curves flatten out. 
- and the code is what, buy how much? keep buying? CAREFUL HERE

- tons of possible features in tickers.tickers['MSFT'].info

- should update the data base every day i guess?
- and trash the old data? because it will get huge?

- is the model unique to each stock? or does the whole sp500 move as one?
'''

"\n- i think i should do a 30 second data step for one day? or a few days? \n- and then i need to 'extract features' to figure out what actualy matters here\n- volume? price jumps? a mix of both? \n- relation to other stocks?\n- and like, does it have to steadily increase for 2 minutes before we buy? make time a variable i can choose\n- also rate of jump in volume and rate of jump in price\n- when to sell? when the curves flatten out. \n- and the code is what, buy how much? keep buying? CAREFUL HERE\n"

In [87]:
# the function is download_and_store_stock_data(ticker, start_date, end_date):

# to avoid the rate limit on this scrape its gonna take a longgggg time

In [125]:
#got rate limited by yfinance to 7 days. so fuck that shit:
def morethan7days(ticker, start_date, end_date):
    print(f"Downloading data for {ticker}...")
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

    while start_date < end_date:
        next_week = start_date + datetime.timedelta(days=7)
        if next_week > end_date:
            next_week = end_date

        print(f"Fetching data from {start_date.date()} to {next_week.date()}")

        # Check if data already exists in the database
        existing_data = list(collection.find({
            'ticker': ticker,
            'Datetime': {
                '$gte': start_date.strftime("%Y-%m-%d %H:%M:%S"),
                '$lt': next_week.strftime("%Y-%m-%d %H:%M:%S")
            }
        }))

        if existing_data:
            print(f"Data for {ticker} from {start_date.date()} to {next_week.date()} already exists in MongoDB.")
        else:
            try:
                data = yf.download(ticker, start=str(start_date.date()), end=str(next_week.date()), interval='1m')
                if not data.empty:
                    # Convert DataFrame to dictionary and add ticker information
                    data_dict = data.reset_index().to_dict(orient="records")
                    for record in data_dict:
                        record['ticker'] = ticker
                    
                    # Store data in MongoDB
                    collection.insert_many(data_dict)
                    print(f"Data for {ticker} from {start_date.date()} to {next_week.date()} stored in MongoDB.")
            except Exception as e:
                print(f"Failed to download data for {ticker}. Error: {str(e)}")
        
        # Update start_date for the next iteration
        start_date = next_week
        
        # Sleep to respect rate limits
        time.sleep(12)  # sleep for 12 seconds

In [134]:
#ALSO DONT ACCIDENTALLY RUN THIS
#for stock in SP500_symbols:
    morethan7days(stock, '2023-10-01', '2023-10-26')


Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for MMM from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for MMM from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for MMM from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for MMM from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for ABT from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for ABT

Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for AKAM from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for AKAM from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for AKAM from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for ALK from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for ALK from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for 

Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for AMZN from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for AMZN from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for AMCR from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for AMCR from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for AMCR from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data fo

Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for AMGN from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for APH from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for APH from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for APH from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for APH from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for AD

Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for AJG from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for AJG from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for AJG from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for AJG from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for AIZ from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for AIZ

...

[*********************100%%**********************]  1 of 1 completed
Data for VRSN from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for VRSN from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for VRSK from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for VRSK from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for VRSK from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for VRSK from 2023-10-22 to 2023-10-26 stored 

[*********************100%%**********************]  1 of 1 completed
Data for WMT from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for WBA from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for WBA from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for WBA from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for WBA from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for DIS from 2023-10-01 to 2023-10-08 stored in Mon

[*********************100%%**********************]  1 of 1 completed
Data for WY from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for WY from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for WY from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for WY from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for WHR from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for WHR from 2023-10-08 to 2023-10-15 stored in MongoDB


1 Failed download:
['ZBH']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2023-10-01 -> 2023-10-08)')



Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for ZBH from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for ZBH from 2023-10-15 to 2023-10-22 stored in MongoDB.
Fetching data from 2023-10-22 to 2023-10-26
[*********************100%%**********************]  1 of 1 completed
Data for ZBH from 2023-10-22 to 2023-10-26 stored in MongoDB.
Fetching data from 2023-10-01 to 2023-10-08
[*********************100%%**********************]  1 of 1 completed
Data for ZION from 2023-10-01 to 2023-10-08 stored in MongoDB.
Fetching data from 2023-10-08 to 2023-10-15
[*********************100%%**********************]  1 of 1 completed
Data for ZION from 2023-10-08 to 2023-10-15 stored in MongoDB.
Fetching data from 2023-10-15 to 2023-10-22
[*********************100%%**********************]  1 of 1 completed
Data for 

# when this is done you should have 477 stocks at minute interval data from october 1 to october 26. then get training

In [93]:
#to view it...

In [115]:
def view_stock_data(stock_symbol):
    # Query the MongoDB database for the specified stock
    query = {"ticker": stock_symbol}
    data = list(collection.find(query))
    
    # Convert the list of dictionaries to a DataFrame
    dbdf = pd.DataFrame(data)
    
    # Dropping the '_id' field that MongoDB automatically adds
    if '_id' in dbdf.columns:
        dbdf = dbdf.drop(columns=['_id'])
    
    return dbdf

In [135]:
# Example usage:
df_msft = view_stock_data('MSFT')
df_msft.head(5)

,Datetime,Open,High,Low,Close,Adj Close,Volume,ticker
0,2023-10-02 13:30:00,316.279999,316.579987,315.549988,315.730011,315.730011,872740,MSFT
1,2023-10-02 13:31:00,315.695007,316.149994,315.179993,315.769989,315.769989,105546,MSFT
2,2023-10-02 13:32:00,315.750000,316.500000,315.709991,316.359985,316.359985,79411,MSFT
3,2023-10-02 13:33:00,316.279999,316.920013,316.179993,316.720001,316.720001,58318,MSFT
4,2023-10-02 13:34:00,316.730011,317.049988,316.649994,317.024994,317.024994,105982,MSFT


In [136]:
df_msft.shape

(7015, 8)

In [137]:
entries_2023_10_25 = df_msft[df_msft['Datetime'].dt.date == datetime.date(2023, 10, 25)]
print('Number of entries for 2023-10-25:', entries_2023_10_25.shape[0])

Number of entries for 2023-10-25: 390


In [138]:
entries_by_hour = df_msft[df_msft['Datetime'].dt.date == datetime.date(2023, 10, 25)].groupby(df_msft['Datetime'].dt.hour).size()
print(entries_by_hour)


Datetime
13    30
14    60
15    60
16    60
17    60
18    60
19    60
dtype: int64


In [123]:
#why are there 120 entries for some hours?

In [139]:
entries_at_2_pm = df_msft[(df_msft['Datetime'].dt.date == datetime.date(2023, 10, 25)) & (df_msft['Datetime'].dt.hour == 14)]
print(entries_at_2_pm)

                Datetime        Open        High         Low       Close  \
6655 2023-10-25 14:00:00  344.630005  344.679993  343.019989  343.019989   
6656 2023-10-25 14:01:00  343.029999  343.470001  342.890015  343.200012   
6657 2023-10-25 14:02:00  343.165009  343.250000  342.559998  342.899994   
6658 2023-10-25 14:03:00  342.893311  343.559998  342.850006  343.410004   
6659 2023-10-25 14:04:00  343.500000  343.510010  343.049988  343.190094   
6660 2023-10-25 14:05:00  343.202515  343.299988  342.920013  343.070007   
6661 2023-10-25 14:06:00  343.079987  343.179993  342.779999  342.890015   
6662 2023-10-25 14:07:00  342.890015  342.890015  342.239990  342.619995   
6663 2023-10-25 14:08:00  342.589996  343.000000  342.589996  342.788513   
6664 2023-10-25 14:09:00  342.790009  343.579987  342.760010  343.579987   
6665 2023-10-25 14:10:00  343.579895  343.579987  342.799988  342.920013   
6666 2023-10-25 14:11:00  342.875092  343.089996  342.730011  342.755005   
6667 2023-10

# mongo db code

In [140]:
#column names
sample_doc = collection.find_one()
print("Columns in the database:")
print(list(sample_doc.keys()))

Columns in the database:
['_id', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ticker']


In [141]:
unique_stocks = collection.distinct('ticker')
print("Number of different stocks:", len(unique_stocks))
print("List of stocks:", unique_stocks)


Number of different stocks: 477
List of stocks: ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DPZ',

In [142]:
num_documents = collection.count_documents({})
print("Number of documents (rows) in the database:", num_documents)

Number of documents (rows) in the database: 3123524


In [143]:
df_amzn = view_stock_data('AMZN')
print("Data for AMZN:")
print(df_amzn.head())

Data for AMZN:
             Datetime        Open        High         Low       Close  \
0 2023-10-02 13:30:00  127.279999  127.349998  126.570000  126.589996   
1 2023-10-02 13:31:00  126.580002  126.769997  126.540001  126.680000   
2 2023-10-02 13:32:00  126.650002  126.959999  126.639999  126.720001   
3 2023-10-02 13:33:00  126.709900  127.000000  126.680000  126.879997   
4 2023-10-02 13:34:00  126.919998  127.180000  126.879997  127.059998   

    Adj Close   Volume ticker  
0  126.589996  1547891   AMZN  
1  126.680000   343080   AMZN  
2  126.720001   259106   AMZN  
3  126.879997   266487   AMZN  
4  127.059998   218266   AMZN  


In [144]:
stats = db.command("dbstats")
print("Data Size: ", stats['dataSize'])
print("Storage Size: ", stats['storageSize'])

Data Size:  447062964.0
Storage Size:  165105664.0


In [146]:
print(447062964.0/(1024*1024))

426.35246658325195


# OK. Sat 10/28 update:
- the database is a half a gig
- 476 or so stocks
- 1 min time interval for the month (ish) of october

# now we train our model?
- on individual stocks?
- and extract features? or something?